# Introduction

We tackle the problem of OCR post processing. In OCR, we map the image form of the document into the text domain. This is done first using an CNN+LSTM+CTC model, in our case based on tesseract. Since this output maps only image to text, we need something on top to validate and correct language semantics.

The idea is to build a language model, that takes the OCRed text and corrects it based on language knowledge. The langauge model could be:
- Char level: the aim is to capture the word morphology. In which case it's like a spelling correction system.
- Word level: the aim is to capture the sentence semnatics. But such systems suffer from the OOV problem.
- Fusion: to capture semantics and morphology language rules. The output has to be at char level, to avoid the OOV. However, the input can be char, word or both.

The fusion model target is to learn:

    p(char | char_context, word_context)

In this workbook we use seq2seq vanilla Keras implementation, adapted from the lstm_seq2seq example on Eng-Fra translation task. The adaptation involves:

- Adapt to spelling correction, on char level
- Pre-train on a noisy, medical sentences
- Fine tune a residual, to correct the mistakes of tesseract 
- Limit the input and output sequence lengths
- Enusre teacher forcing auto regressive model in the decoder
- Limit the padding per batch
- Learning rate schedule
- Bi-directional LSTM Encoder
- Bi-directional GRU Encoder


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from keras.models import load_model
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

/home/ahmad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def calculate_WER_sent(gt, pred):
    '''
    calculate_WER('calculating wer between two sentences', 'calculate wer between two sentences')
    '''
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')
    d = np.zeros(((len(gt_words) + 1), (len(pred_words) + 1)), dtype=np.uint8)
    # d = d.reshape((len(gt_words)+1, len(pred_words)+1))

    # Initializing error matrix
    for i in range(len(gt_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(gt_words) + 1):
        for j in range(1, len(pred_words) + 1):
            if gt_words[i - 1] == pred_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(gt_words)][len(pred_words)]

In [5]:
def calculate_WER(gt, pred):
    '''

    :param gt: list of sentences of the ground truth
    :param pred: list of sentences of the predictions
    both lists must have the same length
    :return: accumulated WER
    '''
#    assert len(gt) == len(pred)
    WER = 0
    nb_w = 0
    for i in range(len(gt)):
        #print(gt[i])
        #print(pred[i])
        WER += calculate_WER_sent(gt[i], pred[i])
        nb_w += len(gt[i])

    return WER / nb_w

In [6]:
def load_data_with_gt(file_name, num_samples, max_sent_len, min_sent_len, delimiter='\t', gt_index=1, prediction_index=0):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []
    gt_texts = []
    target_texts = []
    for row in open(file_name, encoding='utf8'):
        if cnt < num_samples :
            #print(row)
            sents = row.split(delimiter)
            input_text = sents[prediction_index]
            
            target_text = '\t' + sents[gt_index] + '\n'
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len and len(target_text) > min_sent_len and len(target_text) < max_sent_len:
                cnt += 1
                
                input_texts.append(input_text)
                target_texts.append(target_text)
                gt_texts.append(sents[gt_index])
    return input_texts, target_texts, gt_texts

In [7]:
def load_data(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []   
    
    #for row in open(file_name, encoding='utf8'):
    for row in open(file_name):
        if cnt < num_samples :            
            input_text = row           
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len:
                cnt += 1                
                input_texts.append(input_text)
    return input_texts

In [8]:
def vectorize_data(input_texts, max_encoder_seq_length, num_encoder_tokens, vocab_to_int):
    '''Prepares the input text and targets into the proper seq2seq numpy arrays'''
    encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text):
            # c0..cn
            encoder_input_data[i, t, vocab_to_int[char]] = 1.
                
    return encoder_input_data

In [9]:
def decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, int_to_vocab):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, vocab_to_int['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# Load data

# Load model params

In [10]:
data_path = '../../dat/'

In [11]:
vocab_file = 'vocab.npz'
model_file = 'best_model.hdf5'
encoder_model_file = 'encoder_model.hdf5'
decoder_model_file = 'decoder_model.hdf5'

In [12]:
vocab = np.load(file=vocab_file)
vocab_to_int = vocab['vocab_to_int'].item()
int_to_vocab = vocab['int_to_vocab'].item()
max_sent_len = vocab['max_sent_len']
min_sent_len = vocab['min_sent_len']



In [13]:
input_characters = sorted(list(vocab_to_int))
num_decoder_tokens = num_encoder_tokens = len(input_characters) #int(encoder_model.layers[0].input.shape[2])
max_encoder_seq_length = max_decoder_seq_length = max_sent_len - 1#max([len(txt) for txt in input_texts])


In [15]:
num_samples = 1000000
#tess_correction_data = os.path.join(data_path, 'test_data.txt')
#input_texts = load_data(tess_correction_data, num_samples, max_sent_len, min_sent_len)

OCR_data = os.path.join(data_path, 'output_handwritten.txt')
input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len, min_sent_len, delimiter='|',gt_index=0, prediction_index=1)

In [24]:
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])

229
gunshot wound to ardamen
 
 	gunshot wound to abdomen

Csmner
 
 	Conner

Lifting restriotions
 
 	Lifting restrictions

Meofhithey
 
 	Webb, Whitney

soo Venerol Surgery
 
 	M.D. General Surgery

athens
 
 	Athens

Ofr nomkn . tilett
 
 	OFF ALVAREZ File #

Nemours
 
 	Nemours

w gto
 
 	Wilmingto

taientsopp bover
 
 	Paige Koppnhaver



In [16]:
#model.load_weights(model_file)

model = load_model(model_file)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 114)    0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 759808      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 114)    0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

In [18]:
encoder_model = load_model(encoder_model_file)
decoder_model = load_model(decoder_model_file)

/home/ahmad/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [19]:
encoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 114)    0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 759808      input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 512)          0           bidirectional_1[0][2]            
          

In [20]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 114)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 512),  1284096     input_2[0][0]                    
                                                                 input_3[0][0]                    
          

In [21]:


encoder_input_data = vectorize_data(input_texts=input_texts, max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens, vocab_to_int=vocab_to_int)

# Sample output from train data
decoded_sentences = []

for seq_index in range(len(input_texts)):
    # Take one sequence (part of the training set)
    # for trying out decoding.

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    target_text = gt_texts[seq_index]
    decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, int_to_vocab)
    
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('GT sentence:', target_text)
    print('Decoded sentence:', decoded_sentence)   
    decoded_sentences.append(decoded_sentence)
    


-
Input sentence: gunshot wound to ardamen

GT sentence: gunshot wound to abdomen
Decoded sentence: ununurh - Hospate streas

-
Input sentence: Csmner

GT sentence: Conner
Decoded sentence: CLIIDENT

-
Input sentence: Lifting restriotions

GT sentence: Lifting restrictions
Decoded sentence: Lintring should rigros

-
Input sentence: Meofhithey

GT sentence: Webb, Whitney
Decoded sentence: Medithobit

-
Input sentence: soo Venerol Surgery

GT sentence: M.D. General Surgery
Decoded sentence: Sorned Status Dev

-
Input sentence: athens

GT sentence: Athens
Decoded sentence: Date s:

-
Input sentence: Ofr nomkn . tilett

GT sentence: OFF ALVAREZ File #
Decoded sentence: Office Hame available

-
Input sentence: Nemours

GT sentence: Nemours
Decoded sentence: Neprone:

-
Input sentence: w gto

GT sentence: Wilmingto
Decoded sentence: • Wow.

-
Input sentence: taientsopp bover

GT sentence: Paige Koppnhaver
Decoded sentence: Cate note workl ara

-
Input sentence: cniched mnenden

GT sentence: 

-
Input sentence: clsed oi Murkrs

GT sentence: Closed fx R humenus
Decoded sentence: Acsered For Tex:

-
Input sentence: Depleed ae prosmaf end

GT sentence: Displaced fx. proximal end
Decoded sentence: Dedered and eren nger -

-
Input sentence: Rhomows-othice visit

GT sentence: R humenus - Office visit
Decoded sentence: Rhos of werk - 031-325-

-
Input sentence: OEte oisi't

GT sentence: Office visit
Decoded sentence: OtE Motist:

-
Input sentence: Rrohinal end-homers S- night

GT sentence: Proximal end-humerus fx-right
Decoded sentence: Rivital higled Procedure congl.

-
Input sentence: Gentile Dohin

GT sentence: Gentile, John
Decoded sentence: Gencreding Physician

-
Input sentence: Oethosede trooma

GT sentence: Orthopedic trauma
Decoded sentence: Other Ortacis Plist

-
Input sentence: founestevw

GT sentence: Youngstown
Decoded sentence: :obeenser:

-
Input sentence: tmme Cartes

GT sentence: Womens Services
Decoded sentence: Impurter Languages

-
Input sentence: Trsenrapactati

-
Input sentence: Cstine eraenation

GT sentence: Initial evaluation
Decoded sentence: Cottent Indective: 

-
Input sentence: the orymet resulls

GT sentence: Flu OV, MRI results
Decoded sentence: hhe for correncagers in

-
Input sentence: Fennings bittian Bryan

GT sentence: Jennings, William Bryan
Decoded sentence: Innding Seavide stanus:

-
Input sentence: orthopaediy purgery

GT sentence: Orthopaedic Surgery
Decoded sentence: Other of cricked or:

-
Input sentence: hrston Satem

GT sentence: Winston Salem
Decoded sentence: Pristed Date:

-
Input sentence: T mings

GT sentence: JeNNiNgs
Decoded sentence: Tientis:

-
Input sentence: De iheet Alley

GT sentence: Dr. Albert Alley
Decoded sentence: De knee (Dention

-
Input sentence: Flanke Pai

GT sentence: Flank Pain
Decoded sentence: Pain Pat Pline.

-
Input sentence: Tha Pain

GT sentence: Abd Pain
Decoded sentence: Thie Pain.

-
Input sentence: thrme

GT sentence: thru.
Decoded sentence: hhert

-
Input sentence: DrSiiertiey

GT sen

In [22]:
WER_spell_correction = calculate_WER(gt_texts, decoded_sentences)
print('WER_spell_correction |TEST= ', WER_spell_correction)

WER_spell_correction |TEST=  0.18949468085106383


In [23]:
WER_OCR = calculate_WER(gt_texts, input_texts)
print('WER_OCR |TEST= ', WER_OCR)

WER_OCR |TEST=  0.16023936170212766
